# Hint
Kernel of jupyter python crashes when closing the window with "esc". That is a common problem on mac. Use the file in `scripts/object_tracking_with_webcam.py` if you want to use this file on mac.

# Optical Flow & Dense Optical Flow
Optical Flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of the object or the camera.

See documentaiton: https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html

## Assumptions
* pixel intensities of an object do not change between the consecutive frames (e.g. a lightbulb that turns off and on between images will not be tracked)
* neighbouring pixels have similar motion

## How the algorithm works
1. The optical flow methods in OpenCV will take a given set of points in a frame
2. It will then attempt to find these points in the next frame

## Methods: 
### Lucas-Kanade method - https://de.wikipedia.org/wiki/Lucas-Kanade-Methode :
* computes optical flow for a sparse features set (-> points are given)

### Gunner Farnersback's algorithm 
can be used to calculate dense optical flow.

This algorithm calculates the flow for all points in an image.

Only a movement (flow) will be highlighted, everything else will be colored black.

# Lucas-Kanade Method

In [1]:
import numpy as np
import cv2

In [2]:
corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)

In [3]:
# larger motions can be detected by larger winsize but it is more sensitive to noise
# maxLevel is based on pyarmid for image processing: https://en.wikipedia.org/wiki/Pyramid_(image_processing) 
# criteria we use both, speed (count) vs. accuracy of the algorithm (eps)
lucas_kanade_params = dict(winSize=(200,200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)) 

In [4]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 60)
ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# # Points to track
prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

mask = np.zeros_like(prev_frame) # visualization - zeros with the same size of the prev_frame

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculating the optical flow on the actual frame
    next_points, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_points, None, **lucas_kanade_params)

    good_new = next_points[status==1]
    good_prev = prev_points[status==1]

    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        x_new, y_new = new.ravel() # reshape / flattening out an array -> 2d to 1d
        x_prev, y_prev = prev.ravel()

        mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_prev), int(y_prev)), (0,255,0), 3)
        frame = cv2.circle(frame, (int(x_new), int(y_new)), 8, (0,0,255), -1)
    img = cv2.add(frame, mask)
    cv2.imshow("tracking", img)

    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    prev_gray = frame_gray.copy()

    prev_points = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

    

: 

# Dense Optical Flow
The difference to before is, that we now color the points and identify which points are moving.

In [ ]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()

prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cv5tColor(frame2, cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvsImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # flow contains the flow vectors for each pixel
    # it is a 2d array with the same size as the input image

    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True) # magnitude and angle from vertical and horizontal information

    hsv_mask[:,:,0] = ang/2 # that way we can see the direction of the flow and see the angle from 0 to 180

    hsv_mask[:,:,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) # actual value defined by the actual magnitude

    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow("frame", bgr)

    k = cv2.waitKey(10) & 0xFF
    if k == 27: # esc key
        break

    prvsImg = nextImg # update the previous image

cap.release()
cv2.destroyAllWindows()